In [ ]:
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.model_selection import LeaveOneGroupOut, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer
import numpy as np
import matplotlib.pyplot as plt
import random
random.seed(10)

In [ ]:
from common import load_output_data_hydroshoot, load_vertices, load_output_data_wheatfspm, load_output_data_grassleaf, generate_groups_and_masks, nmse

## Load the Hydroshoot data

In [ ]:
fn_data_hydroshoot = "Hydroshoot/gdc_can1_grapevine.csv"
vtc = load_vertices(fn_data_hydroshoot)
vtc_selected = random.sample(vtc, 32)
dfo_hydroshoot = load_output_data_hydroshoot(fn_data_hydroshoot, vtc_selected, "E")
dfi_hydroshoot = pd.read_csv("Hydroshoot/meteo.csv", sep=";")
dfi_hydroshoot.loc[:,"time"] = pd.to_datetime(dfi_hydroshoot.loc[:,"time"])
dfip_hydroshoot = load_output_data_hydroshoot(fn_data_hydroshoot, None, "Ei")

In [ ]:
# find start of the data trace
with open(fn_data_hydroshoot) as f:
    header = f.readline().split(",")
    time_idx = header.index("t")
    data = f.readline().split(",")
    t0 = pd.to_datetime(data[time_idx])

In [ ]:
mask = dfi_hydroshoot["time"] >= t0.to_datetime64()
dfi_hydroshoot = dfi_hydroshoot[mask].reset_index(drop=True)
dfi_hydroshoot = dfi_hydroshoot.drop(index = range(len(dfo_hydroshoot), len(dfi_hydroshoot)))

In [ ]:
for i in dfo_hydroshoot:
    plt.plot(dfo_hydroshoot[i])
plt.show()

In [ ]:
dfo_hydroshoot.reset_index(drop=True, inplace=True)
dfo_hydroshoot.drop(index=range(15*24,int(dfo_hydroshoot.index[-1]+1)),inplace=True)
dfo_hydroshoot.reset_index(drop=True, inplace=True)
dfi_hydroshoot.reset_index(drop=True, inplace=True)
dfi_hydroshoot.drop(index=range(15*24,int(dfi_hydroshoot.index[-1]+1)),inplace=True)
dfi_hydroshoot.reset_index(drop=True, inplace=True)

In [ ]:
dfip_hydroshoot.reset_index(drop=True, inplace=True)
dfip_hydroshoot.drop(index=range(15*24,int(dfip_hydroshoot.index[-1]+1)),inplace=True)
dfip_hydroshoot.reset_index(drop=True, inplace=True)
dfip_hydroshoot = dfip_hydroshoot.to_numpy().sum(axis=1)

In [ ]:
print(f"{len(dfo_hydroshoot)} {len(dfi_hydroshoot)} {len(dfip_hydroshoot)}")

## Load the WheatFspm data

In [ ]:
df1 = pd.read_csv("WheatFSPM/NEMA15/elements_postprocessing.csv")
df2 = pd.read_csv("WheatFSPM/NEMA15/elements_states.csv")
for k in df2.keys():
    if k not in df1:
        df1[k] = df2[k]
df1.to_csv("WheatFspm/NEMA15_dataset.csv", index=False)

In [ ]:
fn_data_wheatfspm = "WheatFSPM/NEMA15_dataset.csv"
dfo_wheatfspm = load_output_data_wheatfspm(fn_data_wheatfspm, "Transpiration")
dfi_wheatfspm = pd.read_csv("WheatFSPM/meteo.csv", index_col="t")
dfip_wheatfspm = load_output_data_wheatfspm(fn_data_wheatfspm, "PARa")
dfi_wheatfspm = dfi_wheatfspm.drop(index = range(len(dfo_wheatfspm), len(dfi_wheatfspm)))

In [ ]:
for i in dfo_wheatfspm:
    plt.plot(dfo_wheatfspm[i])
plt.show()

In [ ]:
dfo_wheatfspm.reset_index(drop=True,inplace=True)
dfo_wheatfspm.drop(index=range(18*24,int(dfo_wheatfspm.index[-1]+1)),inplace=True)
dfo_wheatfspm.reset_index(drop=True,inplace=True)
dfi_wheatfspm.reset_index(drop=True,inplace=True)
dfi_wheatfspm.drop(index=range(18*24,int(dfi_wheatfspm.index[-1]+1)),inplace=True)
dfi_wheatfspm.reset_index(drop=True,inplace=True)

In [ ]:
dfip_wheatfspm.reset_index(drop=True,inplace=True)
dfip_wheatfspm.drop(index=range(18*24,int(dfip_wheatfspm.index[-1]+1)),inplace=True)
dfip_wheatfspm.reset_index(drop=True,inplace=True)
dfip_wheatfspm = dfip_wheatfspm.to_numpy().sum(axis=1)

## Grass Leaf data

In [ ]:
fn_data_grassleaf = "GrassLeaf/model_output_length.csv"
dfo_grassleaf = load_output_data_grassleaf(fn_data_grassleaf)
dfi_grassleaf = pd.read_csv("GrassLeaf/meteo.csv", sep=",")

In [ ]:
for i in dfo_grassleaf:
    plt.plot(dfo_grassleaf[i])
plt.show()

In [ ]:
dfo_grassleaf.drop(index=range(48,11*24),inplace=True)
dfo_grassleaf.reset_index(drop=True, inplace=True)
dfi_grassleaf.drop(index=range(48,11*24),inplace=True)
dfi_grassleaf.reset_index(drop=True, inplace=True)
dfi_grassleaf.drop(index=range(len(dfo_grassleaf), len(dfi_grassleaf)), inplace=True)

In [ ]:
print(f"{len(dfo_grassleaf)} {len(dfi_grassleaf)}")

## Preprocessing

First, we have to make sure that all datasets are of the same length to make the comparison fair.

In [ ]:
sim_len = min([len(dfo_grassleaf), len(dfo_wheatfspm), len(dfo_hydroshoot), 18*24])

In [ ]:
sim_len//24

In [ ]:
#if sim_len < len(dfo_hydroshoot):
#    dfo_hydroshoot = dfo_hydroshoot.drop(index=range(sim_len, len(dfo_hydroshoot)))
#    dfi_hydroshoot = dfi_hydroshoot.drop(index=range(sim_len, len(dfi_hydroshoot)))
#    dfip_hydroshoot = dfip_hydroshoot[:sim_len]
#if sim_len < len(dfo_wheatfspm):
#    dfo_wheatfspm = dfo_wheatfspm.drop(index=range(sim_len, len(dfo_wheatfspm)))
#    dfi_wheatfspm = dfi_wheatfspm.drop(index=range(sim_len, len(dfi_wheatfspm)))
#    dfip_wheatfspm = dfip_wheatfspm[:sim_len]
#if sim_len < len(dfo_grassleaf):
#    dfo_grassleaf = dfo_grassleaf.drop(index=range(sim_len, len(dfo_grassleaf)))
#    dfo_grassleaf = dfo_grassleaf.drop(index=range(sim_len, len(dfo_grassleaf)))

In [ ]:
#dfi_grassleaf = dfi_grassleaf.drop(index = range(len(dfo_grassleaf), len(dfi_grassleaf)))

## Model

Generate model and CV hyperparameter tuning method

## Optimise

Run the model, optimise and report the performance

In [ ]:
for dfo, dfi, dfip, label, y_labels in zip(
        [dfo_hydroshoot, dfo_wheatfspm, dfo_grassleaf],
        [dfi_hydroshoot, dfi_wheatfspm, dfi_grassleaf],
        [dfip_hydroshoot, dfip_wheatfspm, None],
        ["Hydroshoot", "WheatFspm", "GrassLeaf"],
        [("Rg", "Tac"), ("PARi", "air_temperature"), ("Rg", "Tac")]):
    print(f"Running {label} dataset")

    groups, train_mask, test_mask = generate_groups_and_masks(dfo, discard=(22,2), n_day_split=3)

    X = dfo.to_numpy()
    X_train = X[train_mask, :]
    X_test = X[test_mask, :]
    y1 = dfi[y_labels[0]].to_numpy()
    y2 = dfi[y_labels[1]].to_numpy()
    yp = dfip
    y1_train = y1[train_mask]
    y1_test = y1[test_mask]
    y2_train = y2[train_mask]
    y2_test = y2[test_mask]
    yp_train = yp[train_mask] if dfip is not None else None
    yp_test = yp[test_mask] if dfip is not None else None
    groups_train = groups[train_mask]
    groups_test = groups[test_mask]

    for y_train, y_test, target in zip([y1_train, y2_train, yp_train], [y1_test, y2_test, yp_test], ["PAR", "Tair", "PARa"]):

        if y_test is None:
            continue

        search_grid = {"model__alpha": np.logspace(-6,6,100)}
        scorer = make_scorer(nmse, greater_is_better=False)
        pipe = Pipeline([("scaler", StandardScaler(with_mean=True, with_std=True)), ("model", Ridge(fit_intercept=True, solver="svd"))])
        cv = LeaveOneGroupOut()
        grid = GridSearchCV(estimator=pipe, param_grid=search_grid, cv=cv, scoring=scorer, return_train_score=True, refit=True)

        grid.fit(X_train, y_train, groups=groups_train)
        results_df = pd.DataFrame(grid.cv_results_)
        results_df['median_test_score'] = results_df.filter(regex='^split').median(axis=1)
        results_df['rank_test_score'] = results_df['median_test_score'].rank(ascending=False).astype(int)

        pipe.set_params(**results_df.query('rank_test_score == 1')['params'].values[0])

        pipe.fit(X_train, y_train)
        print(f"Optimal hyperparameter value for {target}: {grid.best_params_}")

        t = np.arange(0, len(dfo)) / 24.0
        t = t[test_mask]
        t2 = np.arange(len(t)) / 24.0
        yh_test = pipe.predict(X_test)
        print(f"Performance: {nmse(y_test, yh_test)} (vs) {nmse(y_train, pipe.predict(X_train))}")
        #plt.plot(t2, y_test, label="dataset")
        #plt.plot(t2, yh_test, label="prediction")
        plt.plot(y_test, label="dataset")
        #print(grid.best_estimator_.named_steps['model'].coef_)
        #print(grid.best_estimator_.named_steps['model'].intercept_)
        plt.plot(yh_test, label="prediction")
        plt.title(target)
        plt.legend()
        plt.show()

        df = pd.DataFrame({
            "time": t,
            "time2": t2,
            "y": y_test,
            "yh": yh_test,
        })

        df.to_csv(f"results_{label}-{target}.csv", index=False)


In [ ]:
for dfo, dfi, dfip, label, y_labels in zip(
        [dfo_hydroshoot, dfo_wheatfspm, dfo_grassleaf],
        [dfi_hydroshoot, dfi_wheatfspm, dfi_grassleaf],
        [dfip_hydroshoot, dfip_wheatfspm, None],
        ["Hydroshoot", "WheatFspm", "GrassLeaf"],
        [("Rg", "Tac"), ("PARi", "air_temperature"), ("Rg", "Tac")]):
    print(f"Running {label} dataset")

    groups, train_mask, test_mask = generate_groups_and_masks(dfo, discard=(22,2), n_day_split=3)

    X = dfo.to_numpy()

    y1 = dfi[y_labels[0]].to_numpy()
    y2 = dfi[y_labels[1]].to_numpy()
    yp = dfip
    y1_train = y1[train_mask]
    y1_test = y1[test_mask]
    y2_train = y2[train_mask]
    y2_test = y2[test_mask]
    yp_train = yp[train_mask] if dfip is not None else None
    yp_test = yp[test_mask] if dfip is not None else None
    groups_train = groups[train_mask]
    groups_test = groups[test_mask]

    for y_train, y_test, target in zip([y1_train], [y1_test], ["PAR"]):

        performance_list_y_test = []
        performance_list_y_train = []

        for i in range(X.shape[1]):

            X_train = X[train_mask, :]
            X_train = np.delete(X_train, i, axis=1)
            X_test = X[test_mask, :]
            X_test = np.delete(X_test, i, axis=1)

            search_grid = {"model__alpha": np.logspace(-6,6,100)}
            scorer = make_scorer(nmse, greater_is_better=False)
            pipe = Pipeline([("scaler", StandardScaler(with_mean=True, with_std=True)), ("model", Ridge(fit_intercept=True, solver="svd"))])
            cv = LeaveOneGroupOut()
            grid = GridSearchCV(estimator=pipe, param_grid=search_grid, cv=cv, scoring=scorer, return_train_score=True, refit=True)

            grid.fit(X_train, y_train, groups=groups_train)
            results_df = pd.DataFrame(grid.cv_results_)
            results_df['median_test_score'] = results_df.filter(regex='^split').median(axis=1)
            results_df['rank_test_score'] = results_df['median_test_score'].rank(ascending=False).astype(int)

            pipe.set_params(**results_df.query('rank_test_score == 1')['params'].values[0])

            pipe.fit(X_train, y_train)
            print(f"Optimal hyperparameter value for {target}: {grid.best_params_}")

            t = np.arange(0, len(dfo)) / 24.0
            t = t[test_mask]
            t2 = np.arange(len(t)) / 24.0
            yh_test = pipe.predict(X_test)

            performance_list_y_test.append(nmse(y_test, yh_test))
            performance_list_y_train.append(nmse(y_train, pipe.predict(X_train)))

        df = pd.DataFrame({
            "nmse_train": performance_list_y_train,
            "nmse_test": performance_list_y_test
        })

        df.to_csv(f"error_{label}-{target}.csv", index=False)